In [ ]:
from torch.utils import data
from sklearn.model_selection import train_test_split
import handshape_datasets as hd
import numpy as np
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import handshape_datasets as hd
import numpy as np
import os
from ipynb.fs.full.FID_Pytorch import *
torch.manual_seed(0) # Set for our testing purposes, please do not change!

def show_tensor_images(image_tensor, num_images=16, size=(3, 32, 32), nrow=4, show=True, name='default'):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=nrow)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    #plt.savefig('/home/willys/tesis/Data-augmentation-using-GANs/data_gen/CW-GAN/PugeaultASL_A/'+name)
    if show:
        plt.show()
torch.cuda.empty_cache()

## Hyperparameters

In [ ]:
n_epochs = 70
z_dim = 64
display_step = 50
batch_size = 128
lr = 0.0002
beta_1 = 0.5
beta_2 = 0.999
c_lambda = 10
crit_repeats = 3
device = 'cuda'
save_path = 'numpy_data/PugeaultASL_A/'

## Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, input_dim=10, im_chan=3, hidden_dim=64):
        super(Generator, self).__init__()
        self.input_dim = input_dim
        # Build the neural network
        self.gen = nn.Sequential(
            self.make_gen_block(input_dim, hidden_dim * 8),
            self.make_gen_block(hidden_dim * 8, hidden_dim * 4, kernel_size=4, stride=1),
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2),
            self.make_gen_block(hidden_dim * 2, hidden_dim, stride=1),
            self.make_gen_block(hidden_dim, im_chan, kernel_size=4, final_layer=True),
        )


    def make_gen_block(self, input_channels, output_channels, kernel_size=3, stride=2, final_layer=False):
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.Tanh(),
            )

    def forward(self, noise):
        x = noise.view(len(noise), self.input_dim, 1, 1)
        return self.gen(x)

def get_noise(n_samples, input_dim, device='cpu'):
    return torch.randn(n_samples, input_dim, device=device)


## Discriminator

In [ ]:
class Discriminator(nn.Module):
 
    def __init__(self, im_chan=3, hidden_dim=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            self.make_disc_block(im_chan, hidden_dim),
            self.make_disc_block(hidden_dim, hidden_dim * 2),
            self.make_disc_block(hidden_dim * 2, 1, final_layer=True),
        )

    def make_disc_block(self, input_channels, output_channels, kernel_size=4, stride=2, final_layer=False):
        if not final_layer:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.LeakyReLU(0.2, inplace=True),
            )
        else:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
            )

    def forward(self, image):
        disc_pred = self.disc(image)
        return disc_pred.view(len(disc_pred), -1)

### One_hot_labels

In [ ]:
import torch.nn.functional as F
def get_one_hot_labels(labels, n_classes):
    '''
    Function for creating one-hot vectors for the labels, returns a tensor of shape (?, num_classes).
    Parameters:
        labels: tensor of labels from the dataloader, size (?)
        n_classes: the total number of classes in the dataset, an integer scalar
    '''
    return F.one_hot(labels, num_classes= n_classes)

### Combine_vectors

In [ ]:
def combine_vectors(x, y):
    '''
    Function for combining two vectors with shapes (n_samples, ?) and (n_samples, ?).
    Parameters:
      x: (n_samples, ?) the first vector. 
        In this assignment, this will be the noise vector of shape (n_samples, z_dim), 
        but you shouldn't need to know the second dimension's size.
      y: (n_samples, ?) the second vector.
        Once again, in this assignment this will be the one-hot class vector 
        with the shape (n_samples, n_classes), but you shouldn't assume this in your code.
    '''
    # Note: Make sure this function outputs a float no matter what inputs it receives
    #### START CODE HERE ####
    combined = torch.cat((x.float(),y.float()),1)
    #### END CODE HERE ####
    return combined

## Data Iterator

In [ ]:
class MyTupleDataset(data.Dataset):
    def __init__(self, dataset, batch_size=128):
        super(MyTupleDataset, self).__init__()
        self.x = dataset[0]
        self.y = dataset[1]
        self.len = dataset[0].shape[0]
        self.batch_size = batch_size
        self.act_idx = 0


    def __getitem__(self, index):
        print(index)
        if (index+128 <= self.len):
            x = torch.Tensor(self.x[index:index+128,:,:,:])  # batch dim is handled by the data loader
            y = torch.Tensor(self.y[index:index+128,]) 
        else:
            x2 = torch.Tensor(self.x[index:self.len,:,:,:])  # batch dim is handled by the data loader
            y2 = torch.Tensor(self.y[index:self.len,]) 
            
        return x.permute(0,3, 1, 2), y

    def __len__(self):
        return self.len
    
    def netx(self):
        if (self.act_idx+128 <= self.len):
            x = torch.Tensor(self.x[self.act_idx:(self.act_idx+self.batch_size),:,:,:]) 
            y = torch.Tensor(self.y[self.act_idx:(self.act_idx+self.batch_size),])
            self.act_idx += 128
        else:
            x = torch.Tensor(self.x[self.act_idx:self.len-1,:,:,:]) 
            y = torch.Tensor(self.y[self.act_idx:self.len-1,])
            self.act_idx = self.len
            
        return x.permute(0,3, 1, 2), y
   
    def reset_index(self):
        self.act_idx = 0

## Load Dataset

In [ ]:
def load_dataset():
    data = hd.load('PugeaultASL_A')

    good_min = 40
    good_classes = []
    n_unique = len(np.unique(data[1]['y']))
    for i in range(n_unique):
        images = data[0][np.equal(i, data[1]['y'])]
        if len(images) >= good_min:
            good_classes = good_classes + [i]

    x = data[0][np.in1d(data[1]['y'], good_classes)]
    img_shape = x[0].shape
    print(img_shape)
    y = data[1]['y'][np.in1d(data[1]['y'], good_classes)]
    y_dict = dict(zip(np.unique(y), range(len(np.unique(y)))))
    y = np.vectorize(y_dict.get)(y)
    classes = np.unique(y)
    n_classes = len(classes)

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, train_size=0.8, test_size=0.2, stratify=y)

    #Reduciendo dataset
    reduce = 8
    for i in range(n_classes):
        images = x_train[np.equal(i, y_train)]
        #x_reduce = x_reduce + images[0:(len(images)//2),:,:,:]
        if i==0:
            x_reduce = images[0:(len(images)//reduce),:,:,:]
            y_reduce = np.ones((len(images)//reduce)) * i
        else:
            x_reduce = np.concatenate((x_reduce, images[0:(len(images)//reduce),:,:,:]), axis=0)
            y_temp = np.ones((len(images)//reduce)) * i
            y_reduce = np.concatenate((y_reduce, y_temp), axis=0)

    print('x.shpae: {}, y.shape: {}'.format(x.shape, y.shape))
    print('x_reduce.shpae: {}, y_reduce.shape: {}'.format(x_reduce.shape, y_reduce.shape))

    #Desordeno el nuevo dataset
    shuffler = np.random.permutation(x_reduce.shape[0])
    x_reduce = x_reduce[shuffler]
    y_reduce = y_reduce[shuffler]
    print("dataset final shape: {}, y final:{}".format(x_reduce.shape,y_reduce.shape))


    train_size = x_train.shape[0]
    test_size = x_test.shape[0]

    return n_classes, x_reduce, y_reduce, x_test, y_test 
    

In [ ]:
def get_input_dimensions(z_dim, img_shape, n_classes):
    
    generator_input_dim = z_dim + n_classes
    discriminator_im_chan = img_shape[2] + n_classes

    return generator_input_dim, discriminator_im_chan

### Create Generator and Discriminator with optimizers

In [ ]:
def create_g_and_d(z_dim, img_shape, n_classes, device='cuda', lr=0.0002, beta_1=0.5, beta2=0.999):
    generator_input_dim, discriminator_im_chan = get_input_dimensions(z_dim, img_shape, n_classes)

    gen = Generator(input_dim=generator_input_dim).to(device)
    gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))
    disc = Discriminator(im_chan=discriminator_im_chan).to(device)
    disc_opt = torch.optim.Adam(disc.parameters(), lr=lr, betas=(beta_1, beta_2))

    def weights_init(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            torch.nn.init.normal_(m.weight, 0.0, 0.02)
        if isinstance(m, nn.BatchNorm2d):
            torch.nn.init.normal_(m.weight, 0.0, 0.02)
            torch.nn.init.constant_(m.bias, 0)
    gen = gen.apply(weights_init)
    disc = disc.apply(weights_init)

    return gen, gen_opt, disc, disc_opt

### Get gradient

In [ ]:
def get_gradient(crit, real, fake, epsilon):
    '''
    Return the gradient of the critic's scores with respect to mixes of real and fake images.
    Parameters:
        crit: the critic model
        real: a batch of real images
        fake: a batch of fake images
        epsilon: a vector of the uniformly random proportions of real/fake per mixed image
    Returns:
        gradient: the gradient of the critic's scores, with respect to the mixed image
    '''
    mixed_images = real * epsilon + fake * (1 - epsilon)
    mixed_scores = crit(mixed_images)
    
    gradient = torch.autograd.grad(
        inputs=mixed_images,
        outputs=mixed_scores,
        
        grad_outputs=torch.ones_like(mixed_scores), 
        create_graph=True,
        retain_graph=True,
    )[0]
    return gradient

### Gradient penalty

In [ ]:
def gradient_penalty(gradient):
    '''
    Return the gradient penalty, given a gradient.
    Given a batch of image gradients, you calculate the magnitude of each image's gradient
    and penalize the mean quadratic distance of each magnitude to 1.
    Parameters:
        gradient: the gradient of the critic's scores, with respect to the mixed image
    Returns:
        penalty: the gradient penalty
    '''
    # Flatten the gradients so that each row captures one image
    gradient = gradient.view(len(gradient), -1)

    # Calculate the magnitude of every row
    gradient_norm = gradient.norm(2, dim=1)
    
    # Penalize the mean squared distance of the gradient norms from 1
    penalty = torch.mean((gradient_norm - 1)**2)
    return penalty

### Gen Loss

In [ ]:
def get_gen_loss(crit_fake_pred):
    '''
    Return the loss of a generator given the critic's scores of the generator's fake images.
    Parameters:
        crit_fake_pred: the critic's scores of the fake images
    Returns:
        gen_loss: a scalar loss value for the current batch of the generator
    '''
    gen_loss = -1. * torch.mean(crit_fake_pred)
    return gen_loss

### Crit loss

In [ ]:
def get_crit_loss(crit_fake_pred, crit_real_pred, gp, c_lambda):
    '''
    Return the loss of a critic given the critic's scores for fake and real images,
    the gradient penalty, and gradient penalty weight.
    Parameters:
        crit_fake_pred: the critic's scores of the fake images
        crit_real_pred: the critic's scores of the real images
        gp: the unweighted gradient penalty
        c_lambda: the current weight of the gradient penalty 
    Returns:
        crit_loss: a scalar for the critic's loss, accounting for the relevant factors
    '''
    crit_loss = torch.mean(crit_fake_pred) - torch.mean(crit_real_pred) + c_lambda * gp
    return crit_loss

In [ ]:
def train(gen, gen_opt, disc, disc_opt, dataloader, n_classes, n_epochs=60,
         device='cuda', crit_repeats=3):
    import time
    cur_step = 0
    generator_losses = []
    discriminator_losses = []

    #UNIT TEST NOTE: Initializations needed for grading
    noise_and_labels = False
    fake = False

    fake_image_and_labels = False
    real_image_and_labels = False
    disc_fake_pred = False
    disc_real_pred = False
    #dataloader = MyTupleDataset(dataset)
    for epoch in range(n_epochs):
        print(epoch)
        # Dataloader returns the batches and the labels
        #for real, labels in tqdm(dataloader):
        dataloader.reset_index()
        for i in range((dataloader.len//dataloader.batch_size)+1):
            real, labels = dataloader.netx()
            cur_batch_size = len(real)
            # Flatten the batch of real images from the dataset
            real = real.to(device)

            mean_iteration_critic_loss = 0
            for _ in range(crit_repeats):

                one_hot_labels = get_one_hot_labels(labels.to(torch.int64).to(device), n_classes)
                image_one_hot_labels = one_hot_labels[:, :, None, None]
                image_one_hot_labels = image_one_hot_labels.repeat(1, 1, 32, 32)

                ### Update discriminator ###
                # Zero out the discriminator gradients
                disc_opt.zero_grad()
                # Get noise corresponding to the current batch_size 
                fake_noise = get_noise(cur_batch_size, z_dim, device=device)

                noise_and_labels = combine_vectors(fake_noise, one_hot_labels)
                fake = gen(noise_and_labels)
               
                # Make sure that enough images were generated
                assert len(fake) == len(real)

                fake_image_and_labels = combine_vectors(fake, image_one_hot_labels)
                real_image_and_labels = combine_vectors(real, image_one_hot_labels)
                disc_fake_pred = disc(fake_image_and_labels.detach())
                disc_real_pred = disc(real_image_and_labels)

                # Make sure that enough predictions were made
                assert len(disc_real_pred) == len(real)
                # Make sure that the inputs are different
                assert torch.any(fake_image_and_labels != real_image_and_labels)

                epsilon = torch.rand(len(real), 1, 1, 1, device=device, requires_grad=True)
                gradient = get_gradient(disc,
                                        real_image_and_labels,
                                        fake_image_and_labels.detach(),
                                        epsilon)
                gp = gradient_penalty(gradient)
                disc_loss = get_crit_loss(disc_fake_pred, disc_real_pred, gp, c_lambda)

                # Keep track of the average critic loss in this batch
                mean_iteration_critic_loss += disc_loss.item() / crit_repeats
                # Update gradients
                disc_loss.backward(retain_graph=True)
                # Update optimizer
                disc_opt.step() 

            # Keep track of the average discriminator loss
            discriminator_losses += [mean_iteration_critic_loss]    

            ### Update generator ###
            # Zero out the generator gradients
            gen_opt.zero_grad()

            fake_image_and_labels = combine_vectors(fake, image_one_hot_labels)
            # This will error if you didn't concatenate your labels to your image correctly
            disc_fake_pred = disc(fake_image_and_labels)
            gen_loss = get_gen_loss(disc_fake_pred)
            gen_loss.backward()
            gen_opt.step()

            # Keep track of the generator losses
            generator_losses += [gen_loss.item()]


            if cur_step % display_step == 0 and cur_step > 0:
                #print("FID: {}".format(get_FID(real.cpu(), fake.cpu())))
                #torch.save(gen.state_dict(), 'numpy_data/PugeaultASL_A/iteration_2/gen.state_dict_cur_step_{}'.format(cur_step))
                gen_mean = sum(generator_losses[-display_step:]) / display_step
                disc_mean = sum(discriminator_losses[-display_step:]) / display_step
                print(f"Step {cur_step}: Generator loss: {gen_mean}, discriminator loss: {disc_mean}")
                show_tensor_images(fake)
                show_tensor_images(real)
                step_bins = 20
                x_axis = sorted([i * step_bins for i in range(len(generator_losses) // step_bins)] * step_bins)
                num_examples = (len(generator_losses) // step_bins) * step_bins
                plt.plot(
                    range(num_examples // step_bins), 
                    torch.Tensor(generator_losses[:num_examples]).view(-1, step_bins).mean(1),
                    label="Generator Loss"
                )
                plt.plot(
                    range(num_examples // step_bins), 
                    torch.Tensor(discriminator_losses[:num_examples]).view(-1, step_bins).mean(1),
                    label="Discriminator Loss"
                )
                plt.legend()
                plt.show()
            
            cur_step += 1
            torch.cuda.empty_cache()
            
    return gen

In [ ]:
36987412wG

# TRAIN

In [ ]:
for j in range(10):
    n_classes, x_train, y_train, x_test, y_test = load_dataset()
    img_shape = x_train[0].shape
    print(img_shape)
    
    #Guardo el split de los datos originales
    data_dir = os.path.join(save_path, 'iteration_{}/'.format(j))
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    np.save(data_dir+'x_train',x_train)
    np.save(data_dir+'y_train',y_train)
    np.save(data_dir+'x_test',x_test)
    np.save(data_dir+'y_test',y_test)
            
    x_train = x_train.astype('float32')
    x_train = (x_train - 127.5) / 127.5
    dataloader = MyTupleDataset((x_train,y_train))

    #creamos el generador y discriminador junto con sus opts y losses
    gen, gen_opt, disc, disc_opt = create_g_and_d(z_dim, img_shape, n_classes, 
                                                  device, lr, beta_1, beta_2)

    #realizamos una epoca de entrenamiento
    gen = train(gen, gen_opt, disc, disc_opt, dataloader, n_classes, 380,
                device, crit_repeats)

    #Guardamos el generador entrenado
    torch.save(gen.state_dict(), data_dir+'gen.state_dict')

    #Mergeamos los datos reales con los generados
    '''for i in range(n_classes):
        one_hot_labels = get_one_hot_labels((torch.ones(3000) * i).to(torch.int64).to(device), n_classes)
        fake_noise = get_noise(3000, z_dim, device=device)
        noise_and_labels = combine_vectors(fake_noise, one_hot_labels)
        fake = gen(noise_and_labels)
        image_unflat = fake.detach().cpu()
        final_images = image_unflat.permute(0,2, 3, 1).numpy()

        x_train = np.concatenate((x_train, final_images), axis=0)
        y_tmp = np.ones(3000,)* i
        y_train = np.concatenate((y_train,y_tmp), axis=0)
    
    #Desordeno el nuevo dataset
    shuffler = np.random.permutation(x_train.shape[0])
    x_train = x_train[shuffler]
    y_train = y_train[shuffler]
    print(x_train.shape, y_train.shape) 
    #Guardamos los datos mergeados
    np.save(data_dir+'x_train_aug',((x_train+1)/2)) #para volver a la escala original
    np.save(data_dir+'y_train_aug',y_train)'''

    #Liberamos memoria
    del gen, gen_opt, disc, disc_opt, x_train, y_train, x_test, y_test

In [ ]:
n_classes = 24
img_shape = (32, 32, 3)
for j in range(1):
    data_dir = os.path.join(save_path, 'iteration_{}/'.format(j+2))
    print("save_path: {}".format(data_dir))
    #cargo x_train e y_train
    x_train = np.load(data_dir+'x_train.npy')
    y_train = np.load(data_dir+'y_train.npy')
    print('x_train.shape, y_train.shape')
    print(x_train.shape, y_train.shape) 
    
    for i in range(n_classes):
        #cargo el generador
        gen, _, _, _ = create_g_and_d(z_dim, img_shape, n_classes, device, lr, beta_1, beta_2)
        gen.load_state_dict(torch.load(data_dir+'gen.state_dict_cur_step_10550'))
        gen.eval()
        
        #creo imagenes falsas y las mergeo
        one_hot_labels = get_one_hot_labels((torch.ones(64) * i).to(torch.int64).to(device), n_classes)
        fake_noise = get_noise(64, z_dim, device=device)
        noise_and_labels = combine_vectors(fake_noise, one_hot_labels)
        fake = gen(noise_and_labels)
        image_unflat = fake.detach().cpu()
        final_images = image_unflat.permute(0,2, 3, 1).numpy()
        final_images = final_images * 127.5 + 127.5
        
        x_train = np.concatenate((x_train, final_images), axis=0)
        y_tmp = np.ones(64,)* i
        y_train = np.concatenate((y_train,y_tmp), axis=0)
        
    #Desordeno el nuevo dataset
    shuffler = np.random.permutation(x_train.shape[0])
    x_train = x_train[shuffler]
    y_train = y_train[shuffler]
    print(" dataset final shape: {}, y final:{}".format(x_train.shape,y_train.shape))
    
    #Guardamos los datos mergeados
    x_train = x_train.astype('uint8')
    np.save(data_dir+'x_train_aug',x_train) #para volver a la escala original
    np.save(data_dir+'y_train_aug',y_train)
    
    #limpiamos memoria
    torch.cuda.empty_cache()
    del x_train, y_train, gen

In [ ]:
import torch
inception_model = torch.hub.load('pytorch/vision:v0.9.0', 'inception_v3', pretrained=True)
inception_model.eval()

In [ ]:
fake_features_list = []
real_features_list = []

gen.eval()
n_samples = 512 # The total number of samples
batch_size = 4 # Samples per iteration



cur_samples = 0
x_train = np.load('numpy_data/iteration_20/x_train.npy')
y_train = np.load('numpy_data/iteration_20/y_train.npy')
dataloader = MyTupleDataset((x_train,y_train),512)
real_samples, labels = dataloader.netx()
real_samples = real_samples.detach().cpu()
cur_batch_size = len(real_samples)

one_hot_labels = get_one_hot_labels(labels.to(torch.int64).to(device), 24)
fake_noise = get_noise(cur_batch_size, z_dim, device=device)
noise_and_labels = combine_vectors(fake_noise, one_hot_labels)
fake = gen(noise_and_labels)

FID = get)

In [ ]:
from ipynb.fs.full.FID_Pytorch import *

In [ ]:
real_samples = torch.from_numpy(np.load('numpy_data/iteration_20/x_train.npy').astype('float32'))
fake_samples = torch.from_numpy(np.load('numpy_data/iteration_20/x_train_fake.npy').astype('float32'))

In [ ]:
real_samples = (real_samples - 127.5) / 127.5
fake_samples = (fake_samples - 127.5) / 127.5

In [ ]:
FID = get_FID(real_samples[0:512,:,:,:], fake_samples[0:512,:,:,:])

In [ ]:
FID

In [ ]:
 data = hd.load('PugeaultASL_A')

good_min = 40
good_classes = []
n_unique = len(np.unique(data[1]['y']))
for i in range(n_unique):
    images = data[0][np.equal(i, data[1]['y'])]
    if len(images) >= good_min:
        good_classes = good_classes + [i]

x = data[0][np.in1d(data[1]['y'], good_classes)]
img_shape = x[0].shape
print(img_shape)
y = data[1]['y'][np.in1d(data[1]['y'], good_classes)]
y_dict = dict(zip(np.unique(y), range(len(np.unique(y)))))
y = np.vectorize(y_dict.get)(y)
n_classes = len(classes)

#Reduciendo dataset
for i in range(n_classes):
    images = x[np.equal(i, y)]
    #x_reduce = x_reduce + images[0:(len(images)//2),:,:,:]
    if i==0:
        x_reduce = images[0:(len(images)//2),:,:,:]
        y_reduce = np.ones((len(images)//2)) * i
    else:
        x_reduce = np.concatenate((x_reduce, images[0:(len(images)//2),:,:,:]), axis=0)
        y_temp = np.ones((len(images)//2)) * i
        y_reduce = np.concatenate((y_reduce, y_temp), axis=0)
        
print('x.shpae: {}, y.shape: {}'.format(x.shape, y.shape))
print('x_reduce.shpae: {}, y_reduce.shape: {}'.format(x_reduce.shape, y_reduce.shape))

#Desordeno el nuevo dataset
shuffler = np.random.permutation(x_reduce.shape[0])
x_reduce = x_reduce[shuffler]
y_reduce = y_reduce[shuffler]
print("dataset final shape: {}, y final:{}".format(x_reduce.shape,y_reduce.shape))


x_train, x_test, y_train, y_test = train_test_split(
    x_reduce, y_reduce, train_size=0.8, test_size=0.2, stratify=y_reduce)

classes = np.unique(y_train)
#n_classes = len(classes)
train_size = x_train.shape[0]
test_size = x_test.shape[0]

In [ ]:
[np.in1d(data[1]['y'], good_classes)][0].shape

In [ ]:
x_reduce = np.array([32,32,3])

In [ ]:
x_reduce.shape

In [ ]:
x.shape

In [ ]:
del x_train, x_test, y_train, y_test

In [ ]:
x_train.shape

In [ ]:
data_dir

In [ ]:
x_reduce_aug = np.load('numpy_data/PugeaultASL_A/iteration_2/x_train_aug.npy')
x_test = np.load('numpy_data/PugeaultASL_A/iteration_1/x_test.npy')

x_reduce_aug = (x_reduce_aug.astype('float32') -127.5 ) / 127.5
x_test = (x_test.astype('float32') -127.5 ) / 127.5

In [ ]:
get_FID(torch.from_numpy(x_test[0:512]),torch.from_numpy(x_reduce_aug[0:512]))

In [ ]:
x_test.dtype